1. Grid Search
2. Try LogReg with All Parameters


In [1]:
import re
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline



In [2]:
data_path_raw = Path.cwd().parent.parent / "data" / "raw"
data_path_preprocessed = Path.cwd().parent.parent / "data" / "processed"

In [3]:
raw_df_processed = pd.read_csv(data_path_preprocessed / "training_set_features_encoded_imputed_standardized.csv", index_col="respondent_id")
labels_df = pd.read_csv(data_path_raw / "training_set_labels.csv", index_col="respondent_id")
test_df_processed = pd.read_csv(data_path_preprocessed / "test_set_features_encoded_imputed_standardized.csv", index_col="respondent_id")

all_raw_df = raw_df_processed.join(labels_df)

In [4]:
import time 

from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [5]:
#3 
X_train, X_test, y_train, y_test = train_test_split(
        raw_df_processed,
        labels_df,
        shuffle = True,
        test_size = 0.25,
        random_state = 10)


In [6]:
pipeline = Pipeline([
            ('LogReg', MultiOutputClassifier(LogisticRegression())) 
                 ])

In [ ]:
from sklearn.model_selection import GridSearchCV

param = {
    'LogReg__estimator__penalty' : ['l1', 'l2' ,'elasticnet', 'none'],
    'LogReg__estimator__dual' : [False, True],
    'LogReg__estimator__C' : [0.5,1.,2.,5.],
    'LogReg__estimator__fit_intercept' : [False, True], 
    'LogReg__estimator__class_weight' :['balanced', 'None'],
    'LogReg__estimator__solver' : ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'LogReg__estimator__max_iter' : [100],
    'LogReg__estimator__l1_ratio' : [0, 0.5, 1]
    }    

clf = GridSearchCV(estimator=pipeline, param_grid = param, cv = 5, n_jobs=-1)
clf.fit(raw_df_processed, labels_df)




In [ ]:
clf.best_params_

#{'LogReg__estimator__C': 0.5,
 #'LogReg__estimator__class_weight': 'None',
 #'LogReg__estimator__dual': False,
 #'LogReg__estimator__fit_intercept': True,
 #'LogReg__estimator__l1_ratio': 0,
 #'LogReg__estimator__max_iter': 1000,
 #'LogReg__estimator__penalty': 'l1',
 #'LogReg__estimator__solver': 'saga'}

In [ ]:
results = pd.DataFrame(clf.cv_results_)
results[results['rank_test_score'] < 10]
top_results = results[results['rank_test_score'] < 10].sort_values(ascending = True, by = 'rank_test_score')
top_results

In [ ]:
#clf.cv_results_['params'][404]

In [ ]:
logisticRegr = MultiOutputClassifier(LogisticRegression(
                        C = 0.5,
                        class_weight = None,
                        dual = False,
                        fit_intercept = True,
                        max_iter = 1000,
                        penalty = 'l1', 
                        solver = 'saga'))

logisticRegr.fit(X_train, y_train)
test_probability = logisticRegr.predict_proba(X_test)


In [ ]:
y_preds = pd.DataFrame(
{
        "h1n1_vaccine": test_probability[0][:, 1],
        "seasonal_vaccine": test_probability[1][:, 1],
    },
    index = y_test.index
    )

In [ ]:
from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_test, y_preds))

In [ ]:
logisticRegr = MultiOutputClassifier(LogisticRegression(
                        C = 0.5,
                        class_weight = None,
                        dual = False,
                        fit_intercept = True,
                        max_iter = 1000,
                        penalty = 'l1', 
                        solver = 'saga'))

logisticRegr.fit(raw_df_processed, labels_df)
test_probability = logisticRegr.predict_proba(test_df_processed)

In [ ]:
predictions = pd.DataFrame(
    {
        "h1n1_vaccine": test_probability[0][:, 1],
        "seasonal_vaccine": test_probability[1][:, 1],
    },
    index = test_df_processed.index
)


In [ ]:
submission_df = pd.read_csv(data_path_raw / "submission_format.csv", 
                            index_col="respondent_id")

In [ ]:
np.testing.assert_array_equal(test_df_processed.index.values, 
                              submission_df.index.values)


In [ ]:
# Save predictions to submission data frame
submission_df["h1n1_vaccine"] = predictions["h1n1_vaccine"]
submission_df["seasonal_vaccine"] = predictions["seasonal_vaccine"]

In [ ]:
#CHANGE PATH 

output_path = Path.cwd().parent.parent / "models" / "submissions"


submission_df.to_csv(output_path /'logreg_JFR_200729__Proproccessed__GridSearch.csv', index=True)

